# Notebook 01: Ingesta de Datos

**Hito 1 - Análisis de Redes y Consumo de Servicios**

Este notebook realiza:
1. Carga de configuración desde `config.yaml`
2. Detección y lectura del dataset crudo desde `data/raw/`
3. Profiling inicial (shapes, tipos, nulos)
4. Guardado de snapshot para trazabilidad

## 1. Importación de Librerías

In [8]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import warnings
import importlib
warnings.filterwarnings('ignore')

# Añadir src/ al path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

# Limpiar módulos cargados anteriormente si existen
modules_to_reload = ['config_loader', 'logging_setup', 'io_utils', 'validate', 'cleaning', 'network_prep', 'eda_basic']
for mod in modules_to_reload:
    if mod in sys.modules:
        del sys.modules[mod]

# Importar módulos del proyecto
import config_loader
import logging_setup
import io_utils
import validate
import cleaning
import network_prep
import eda_basic

# Imports específicos para usar directamente
from config_loader import load_config, get_absolute_path
from logging_setup import get_etl_logger, log_section
from io_utils import find_data_file, read_data_file, write_data_file, get_data_snapshot_path
from io_utils import profile_dataframe, print_profile

print("✓ Librerías importadas correctamente")

✓ Librerías importadas correctamente


## 2. Carga de Configuración

In [9]:
# Cargar configuración
config = load_config()

# Inicializar logger
logger = get_etl_logger(config)
log_section(logger, "NOTEBOOK 01: INGESTA DE DATOS")

print("✓ Configuración cargada")
print(f"  Rutas configuradas: {list(config['paths'].keys())}")
print(f"  Columnas esperadas: {config['columns_expected']}")
print(f"  Semilla aleatoria: {config['random_seed']}")

2025-10-27 23:28:23 | INFO     | etl_pipeline | ================================================================================
2025-10-27 23:28:23 | INFO     | etl_pipeline |   NOTEBOOK 01: INGESTA DE DATOS
2025-10-27 23:28:23 | INFO     | etl_pipeline | ================================================================================
✓ Configuración cargada
  Rutas configuradas: ['data_raw', 'data_processed', 'figures', 'reports', 'notebooks']
  Columnas esperadas: ['PERSONA', 'TIPO DE SERVICIO', 'NOMBRE DE LA TAREA', 'AÑO', 'MODALIDAD', 'COMPLEJIDAD']
  Semilla aleatoria: 42
2025-10-27 23:28:23 | INFO     | etl_pipeline |   NOTEBOOK 01: INGESTA DE DATOS
2025-10-27 23:28:23 | INFO     | etl_pipeline | ================================================================================
✓ Configuración cargada
  Rutas configuradas: ['data_raw', 'data_processed', 'figures', 'reports', 'notebooks']
  Columnas esperadas: ['PERSONA', 'TIPO DE SERVICIO', 'NOMBRE DE LA TAREA', 'AÑO', 'MODALIDAD'

## 3. Detección y Lectura del Dataset Crudo

In [10]:
# Detectar archivo en data/raw/
raw_path = get_absolute_path(config, 'data_raw')
data_file = find_data_file(raw_path)

logger.info(f"Archivo detectado: {data_file}")
print(f"📁 Archivo a procesar: {data_file.name}")

2025-10-27 23:28:23 | INFO     | etl_pipeline | Archivo detectado: /Users/kanyewest/Documents/ComplexNetworks/ComplexNetworks_TP/data/raw/datosRed.xlsx
📁 Archivo a procesar: datosRed.xlsx
📁 Archivo a procesar: datosRed.xlsx


In [11]:
# Leer el dataset
data = read_data_file(data_file, expected_columns=config['columns_expected'])

logger.info(f"Datos cargados: {data.shape[0]} filas, {data.shape[1]} columnas")
print(f"✓ Datos cargados: {data.shape}")

2025-10-27 23:28:23 | INFO     | etl_pipeline | Datos cargados: 10384 filas, 6 columnas
✓ Datos cargados: (10384, 6)
✓ Datos cargados: (10384, 6)


## 4. Profiling Inicial

In [12]:
# Vista rápida de los datos
print("Primeras filas:")
display(data.head())

print("\n" + "="*60)
data.info()

Primeras filas:


,AÑO,PERSONA,TIPO DE SERVICIO,NOMBRE DE LA TAREA,MODALIDAD,COMPLEJIDAD
0,2019,ID_1,SERV_1,TAREA_1.1,PRESENCIAL,BAJA
1,2019,ID_2,SERV_1,TAREA_1.1,PRESENCIAL,BAJA
2,2019,ID_3,SERV_1,TAREA_1.1,PRESENCIAL,BAJA
3,2019,ID_4,SERV_1,TAREA_1.1,PRESENCIAL,BAJA
4,2019,ID_5,SERV_1,TAREA_1.1,PRESENCIAL,BAJA



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10384 entries, 0 to 10383
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   AÑO                 10384 non-null  int64 
 1   PERSONA             10384 non-null  object
 2   TIPO DE SERVICIO    10384 non-null  object
 3   NOMBRE DE LA TAREA  10384 non-null  object
 4   MODALIDAD           10384 non-null  object
 5   COMPLEJIDAD         10384 non-null  object
dtypes: int64(1), object(5)
memory usage: 486.9+ KB


In [13]:
# Profiling detallado
profile = profile_dataframe(data)
print_profile(profile)

logger.info(f"Profiling completado: {profile['n_rows']} filas, {profile['duplicated_rows']} duplicados")

  Filas: 10,384
  Columnas: 6
  Filas duplicadas: 2429
  Memoria: 3.25 MB

  Columnas y tipos:
    - AÑO: int64 (nulos: 0, 0.0%)
    - PERSONA: object (nulos: 0, 0.0%)
    - TIPO DE SERVICIO: object (nulos: 0, 0.0%)
    - NOMBRE DE LA TAREA: object (nulos: 0, 0.0%)
    - MODALIDAD: object (nulos: 0, 0.0%)
    - COMPLEJIDAD: object (nulos: 0, 0.0%)
2025-10-27 23:28:24 | INFO     | etl_pipeline | Profiling completado: 10384 filas, 2429 duplicados


## 5. Guardado de Snapshot para Trazabilidad

In [14]:
# Generar ruta de snapshot
processed_path = get_absolute_path(config, 'data_processed')
snapshot_path = get_data_snapshot_path(processed_path / data_file.name, suffix=config['outputs']['snapshot_suffix'])

# Guardar snapshot
write_data_file(data, snapshot_path)

logger.info(f"Snapshot guardado: {snapshot_path}")
print(f"✓ Snapshot guardado: {snapshot_path.name}")
print(f"\n{'='*60}")
print(f"INGESTA COMPLETADA")
print(f"{'='*60}")

2025-10-27 23:28:24 | INFO     | etl_pipeline | Snapshot guardado: /Users/kanyewest/Documents/ComplexNetworks/ComplexNetworks_TP/data/processed/datosRed_snapshot.csv
✓ Snapshot guardado: datosRed_snapshot.csv

INGESTA COMPLETADA
✓ Snapshot guardado: datosRed_snapshot.csv

INGESTA COMPLETADA
